# Coversational Interface - Chatbot with Claude LLM
In this notebook, we will build a chatbot using the Foundation Models (FMs) in Amazon Bedrock. For our use-case we use Amazon Nova Lite as our FM for building the chatbot and InMemoryChatMessageHistory to store the conversation history.

## Overview

Conversational interfaces such as chatbots and virtual assistants can be used to enhance the user experience for your customers. Chatbots uses natural language processing (NLP) and machine learning algorithms to understand and respond to user queries. Chatbots can be used in a variety of applications, such as customer service, sales, and e-commerce, to provide quick and efficient responses to users. They can be accessed through various channels such as websites, social media platforms, and messaging apps.

![Using Amazon Bedrock to support a multi-turn conversation with AI](.././images/chatbot_bedrock.png)


In [ ]:
!pip install --upgrade -q -r requirements.txt

In [13]:
# Restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [14]:
import boto3
import botocore

In [15]:
from langchain_aws.chat_models import ChatBedrock
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory

### Set up

In [16]:
boto3_session = boto3.session.Session()
region = boto3_session.region_name

# the statement below can be used to override the region in the session
#region = "us-west-2"

### Run the chatbot

Set up parameters for the model and create a client

In [17]:
model = "us.amazon.nova-lite-v1:0"
temperature = 0.1

In [18]:
llm_chat = ChatBedrock(
    model_id=model, 
    model_kwargs={"temperature": temperature},
    region_name=region
)


Passing conversation state into and out a chain is vital when building a chatbot. The [RunnableWithMessageHistory class](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html) lets us add message history to certain types of chains. It wraps another Runnable and manages the chat message history for it. Specifically, it loads previous messages in the conversation BEFORE passing it to the Runnable, and it saves the generated response as a message AFTER calling the runnable. This class also enables multiple conversations by saving each conversation with a session_id - it then expects a session_id to be passed in the config when calling the runnable, and uses that to look up the relevant conversation history. You have to implement a function like the get_session_history() which will take as input the session-id and return the conversation for the session. Here we have a simple implementation of the conversation history, using the InMemoryChatMessageHistory class

In [19]:
store = {}

def get_session_history(session_id):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(llm_chat, 
                                                  get_session_history)


In [20]:
from langchain_core.messages import HumanMessage

response = with_message_history.invoke(
    [HumanMessage(content="hi - i am bob!")],
    config={"configurable": {"session_id": "1"}},
)
response.content

"Hello, Bob! It's great to meet you. If there's anything you'd like to talk about or if you have any questions, feel free to let me know. Whether it's about a specific topic, a problem you're facing, or just a casual chat, I'm here to help. How can I assist you today?"

In [21]:
response = with_message_history.invoke(
    [HumanMessage(content="whats my name?")],
    config={"configurable": {"session_id": "1"}},
)
response.content

"Your name is Bob! If you have any questions or need assistance with anything, feel free to ask. Whether it's about a topic you're interested in, a problem you're facing, or just a casual conversation, I'm here to help. How can I assist you today, Bob?"

At this point the store has 1 key (session_id = '1') and its value is a list with 4 messages:
 [HumanMessage, AIMessage, HumanMessage, AIMessage]

All langchain messages have 3 properties: a role, content, response_metadata. The HumanMessage returns 1 property, the *content* (e.g. the message that the user passed) whereas the AIMessage also returns non-empty *response_metadata*. It also includes the property *usage_metadata*, a dictionary with these keys: input_tokens, output_tokens, total_tokens (these are also included in the response_metadata) 


In [22]:
print(store)
# uncomment the following line to take a closer look at the message associated with session_id='1'
#store['1'].messages[1]

{'1': InMemoryChatMessageHistory(messages=[HumanMessage(content='hi - i am bob!', additional_kwargs={}, response_metadata={}), AIMessage(content="Hello, Bob! It's great to meet you. If there's anything you'd like to talk about or if you have any questions, feel free to let me know. Whether it's about a specific topic, a problem you're facing, or just a casual chat, I'm here to help. How can I assist you today?", additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': '15bb0cee-c2b7-40aa-8eb3-f80ea860ef57', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Mon, 02 Jun 2025 09:37:22 GMT', 'content-type': 'application/json', 'content-length': '447', 'connection': 'keep-alive', 'x-amzn-requestid': '15bb0cee-c2b7-40aa-8eb3-f80ea860ef57'}, 'RetryAttempts': 0}, 'stopReason': 'end_turn', 'metrics': {'latencyMs': [454]}, 'model_name': 'us.amazon.nova-lite-v1:0'}, id='run--ffb37d51-256b-4dcb-b6c3-83ecea6a2f1a-0', usage_metadata={'input_tokens': 6, 'output_tokens': 74, 'total_tok

### Create a Multi-Lingual Greeter Chatbot! 
Building on our above example the RunnableWithMessageHistory class from langchain which serves to:
1) retain `InMemoryChatMessageHistory` for each follow up message within the same session ID
2) call the LLM ([*Runnable*](https://python.langchain.com/v0.1/docs/expression_language/interface/))
3) write the AI response message back into the `InMemoryChatMessageHistory`

Now we will introduce [ChatPromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html) to configure our chat application to be multi-lingual!

In [23]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system","You're an assistant who speaks in {language}. Translate the user input"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])

chain = prompt | llm_chat

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
)

lang = "French"
print(chain_with_history.invoke(
    {"language": lang, "question": "Hi my name is John"},
    config={"configurable": {"session_id": "2"}}
))

content="Bonjour, je m'appelle John." additional_kwargs={} response_metadata={'ResponseMetadata': {'RequestId': 'ab99e96b-275b-4f29-9630-c7d75e20d8c5', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Mon, 02 Jun 2025 09:39:08 GMT', 'content-type': 'application/json', 'content-length': '207', 'connection': 'keep-alive', 'x-amzn-requestid': 'ab99e96b-275b-4f29-9630-c7d75e20d8c5'}, 'RetryAttempts': 0}, 'stopReason': 'end_turn', 'metrics': {'latencyMs': [211]}, 'model_name': 'us.amazon.nova-lite-v1:0'} id='run--41161310-cd3c-453e-8de2-4fc32fadc54f-0' usage_metadata={'input_tokens': 19, 'output_tokens': 8, 'total_tokens': 27, 'input_token_details': {'cache_creation': 0, 'cache_read': 0}}


In [24]:
print(chain_with_history.invoke(
    {"language": lang, "question": "What is my name?"},
    config={"configurable": {"session_id": "2"}}
))

content='Ton nom est John.' additional_kwargs={} response_metadata={'ResponseMetadata': {'RequestId': '5ec07356-9e81-4488-80f5-041ee763230b', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Mon, 02 Jun 2025 09:39:15 GMT', 'content-type': 'application/json', 'content-length': '197', 'connection': 'keep-alive', 'x-amzn-requestid': '5ec07356-9e81-4488-80f5-041ee763230b'}, 'RetryAttempts': 0}, 'stopReason': 'end_turn', 'metrics': {'latencyMs': [200]}, 'model_name': 'us.amazon.nova-lite-v1:0'} id='run--9a8d6c26-4228-4c04-ba9e-e6382380c3df-0' usage_metadata={'input_tokens': 37, 'output_tokens': 5, 'total_tokens': 42, 'input_token_details': {'cache_creation': 0, 'cache_read': 0}}


In [25]:
print(store['2'].messages)

[HumanMessage(content='Hi my name is John', additional_kwargs={}, response_metadata={}), AIMessage(content="Bonjour, je m'appelle John.", additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': 'ab99e96b-275b-4f29-9630-c7d75e20d8c5', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Mon, 02 Jun 2025 09:39:08 GMT', 'content-type': 'application/json', 'content-length': '207', 'connection': 'keep-alive', 'x-amzn-requestid': 'ab99e96b-275b-4f29-9630-c7d75e20d8c5'}, 'RetryAttempts': 0}, 'stopReason': 'end_turn', 'metrics': {'latencyMs': [211]}, 'model_name': 'us.amazon.nova-lite-v1:0'}, id='run--41161310-cd3c-453e-8de2-4fc32fadc54f-0', usage_metadata={'input_tokens': 19, 'output_tokens': 8, 'total_tokens': 27, 'input_token_details': {'cache_creation': 0, 'cache_read': 0}}), HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}), AIMessage(content='Ton nom est John.', additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': '5e